In [6]:
# read the data
#imports"
from pathlib import Path
from rdkit import Chem
from rdkit.Chem import Draw

# Create a Path object for the current directory
current_directory = Path.cwd()
print("Current Directory:", current_directory.resolve())

# Creating a Path object for an example file that does not yet exist
example_file_path = current_directory / "database.csv"

# Reading the contents of the file to test if we have well acess to our data
if example_file_path.exists():
    with example_file_path.open("r") as file:
        first_line = file.readline()
        print(first_line)
else:
    print("The file does not exist.")

Current Directory: C:\Users\33783\Documents\EPFL\Etudes\BA4\Amazing-Project
Reactant 1;Reactant 2;Product 1;Product 2;Product 3



In [7]:
# function to convert name into SMILES
!pip install pubchempy

import pubchempy as pcp

def name_to_smiles(molecule_name):
    """
    Convert a molecule name to a SMILES notation using PubChemPy's PubChem database.
    Args:
    - molecule_name (str): The name of the molecule.
    Returns:
    - smiles (str): The SMILES notation of the molecule, or None if retrieval fails.
    """
    try:
        compound = pcp.get_compounds(molecule_name, 'name')
        if compound:
            return compound[0].canonical_smiles
        else:
            print("Error: Unable to retrieve molecule information. Please try with the SMILE of the molecule")
            return None
    except:
        print("Error: Unable to retrieve molecule information. Please try again")
        return None


In [8]:
# function to determine if the user enters the SMILES notation or the usual name of the molecule
from rdkit import Chem

def is_smiles(smiles):
    """
    Check if a string represents a valid SMILES notation.
    Args:
    - smiles (str): The string to check.
    Returns:
    - is_valid (bool): True if the string is a valid SMILES notation, False otherwise.
    """
    if Chem.MolFromSmiles(smiles)== None:
        return False
    else:
        return True

# Check if the entered string is a valid SMILES notation
if is_smiles(string_input):
    print(f"'{string_input}' is a valid SMILES notation.")
else:
    print(f"'{string_input}' is not a valid SMILES notation.")

NameError: name 'string_input' is not defined

In [11]:
# Prompt the user to enter the molecule that he wants to form
string_input = input("Enter the name of the molecule: ")

Enter the name of the molecule:  phenantroline


In [12]:
# Call the function to convert the molecule name to SMILES notation
if is_smiles(string_input)== False:
    smiles = name_to_smiles(string_input)
    if smiles:
        print(f"SMILES notation for {string_input}: {smiles}")
else:
    print (string_input)

[15:55:54] SMILES Parse Error: syntax error while parsing: phenantroline
[15:55:54] SMILES Parse Error: Failed parsing SMILES 'phenantroline' for input: 'phenantroline'


SMILES notation for phenantroline: C1=CC2=C(C3=C(C=CC=N3)C=C2)N=C1


In [ ]:
# fonction to remove atome mapping numbers from smiles
import re

def remove_atom_mapping(smiles):
    """
    Remove atom mapping numbers from a SMILES-like notation.

    Args:
    - smiles (str): The SMILES-like notation with atom mapping numbers.

    Returns:
    - smiles_without_mapping (str): The SMILES notation without atom mapping numbers.
    """
    # Define a regular expression pattern to match atom mapping numbers
    pattern = r':\d+'

    # Replace atom mapping numbers with an empty string
    smiles_without_mapping = re.sub(pattern, '', smiles)

    return smiles_without_mapping 


In [ ]:
# print the yield

In [ ]:
# Smiles with mapping to without maping
smiles_with_mapping = "[Br-:1].[Br:2][C:5]([C:4]([H:3])([C:31]#[N:32])[C:41]#[N:42])([H:11])[H:21]"
clean_smiles = remove_atom_mapping(smiles_with_mapping)
print("SMILES without atom mapping numbers:", clean_smiles)

In [9]:
# print the molecule
original_smiles='[Br:1][C:5]([C:4]([H:3])([C:31]#[N:32])[C:41]#[N:42])([H:11])[H:21]'
# Parse the original SMILES notation
mol = Chem.MolFromSmiles(original_smiles)
# Convert the parsed molecule back to SMILES without atom mapping numbers
smiles_without_numbers = Chem.MolToSmiles(mol, allBondsExplicit=True, isomericSmiles=True)


# Create a molecule object from the SMILES notation
mol = Chem.MolFromSmiles(smiles_without_numbers)

# Draw and display the molecule
img = Draw.MolToImage(mol)

# Display the image
img.show()